In [1]:
import keras
import numpy as np
import tensorflow as tf
from tqdm import tqdm


Using TensorFlow backend.


In [3]:
%conda install regex

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::pandas==1.0.1=py36h0573a6f_0
  - defaults/noarch::jupyterlab==1.2.6=pyhf63ae98_0
  - defaults/linux-64::scikit-learn==0.22.1=py36hd81dba3_0
  - defaults/linux-64::python-language-server==0.31.7=py36_0
  - defaults/linux-64::bkcharts==0.2=py36_0
  - defaults/linux-64::nb_conda==2.2.1=py36_0
  - defaults/noarch::numpydoc==0.9.2=py_0
  - defaults/linux-64::pytest-arraydiff==0.3=py36h39e3cac_0
  - defaults/linux-64::bottleneck==1.3.2=py36heb32a55_0
  - defaults/noarch::sphinx==2.4.0=py_0
  - defaults/linux-64::pywavelets==1.1.1=py36h7b6447c_0
  - defaults/noarch::pytest-astropy==0.8.0=py_0
  - defaults/linux-64::numexpr==2.7.1=py36h423224d_0
  - defaults/noarch::anaconda-project==0.8.4=py_0
  - defaults/linux-64::nbconvert==5.6.1=py36_0
  - defaults/linux-64::h5py==2.10.0=py36h7918eee_0
  - defaults/linux-64::bokeh==1

# Prepare data

In [2]:
import unicodedata
import regex as re

def normalize_text(text):
    clean_text=text.lower()
    clean_text=re.sub(r"[^a-zA-Z0-9\säåö]"," ",clean_text)
    #clean_text=re.sub(r"[^\S\n]+"," ",clean_text)
    clean_text=re.sub(r"\s+"," ",clean_text)
    clean_text=re.sub(r"^\s","",clean_text)
    clean_text=re.sub(r"\s$","",clean_text)
    return clean_text

normalize_text("äåöjkjh jhjk j1hkj"),normalize_text("jFjh.jhjk,.,.ef jhkj "),normalize_text(" jkjh jhjk jhkj")

('äåöjkjh jhjk j1hkj', 'jfjh jhjk ef jhkj', 'jkjh jhjk jhkj')

In [3]:
import json 
import random
finnish_companies = json.load(open("finnish_kaggle_companies.json","r+"))
all_companies = json.load(open("all_kaggle_companies.json","r+"))

# companies 
names_arr=[normalize_text(company) for company in tqdm(all_companies)]
random.shuffle(names_arr)
names_text="\n".join(names_arr)

# finnish 
finnish_names_arr=[normalize_text(company) for company in tqdm(finnish_companies)]
random.shuffle(finnish_names_arr)
finnish_names_text="\n".join(finnish_names_arr)


characters=sorted(list(set(names_text+finnish_names_text)))

char_indices = dict((c, i) for i, c in enumerate(characters))
indices_char = dict((i, c) for i, c in enumerate(characters))

Tx=25


100%|██████████| 17842/17842 [00:00<00:00, 54445.48it/s]


In [4]:
print("Characters: ","".join(characters))
print("Chars: ",len(characters))
print("Tx: ",Tx)

Characters:  
 0123456789abcdefghijklmnopqrstuvwxyzäåö
Chars:  41
Tx:  25


In [34]:
def str_to_vec(word, start_with_null=False):
    """
    Converts word to vec
    
    word -- string
    
    returns array of shape (Tx, len(chars))
    """
    if start_with_null:
        word="\n"+word
    x = np.zeros((len(word), len(characters)), dtype=np.bool)
    for t, char in enumerate(word):
        x[t, char_indices[char]] = 1
    return x
    
def vec_to_str(vec):
    """
    Converts vec to word
    
    vec -- array of shape (Tx, len(chars))
    
    """
    word=""
    for i in range(vec.shape[0]):
        word+=indices_char[np.argmax(vec[i])]
    return word

a=str_to_vec("hello world",start_with_null=True)
#print(a)
print(vec_to_str(a))


hello world


In [8]:
def vectorization(text, stride, n_x, Tx):
    """
    Convert X and Y (lists) into arrays to be given to a recurrent neural network.
    
    Arguments:
    X -- 
    Y -- 
    Tx -- integer, sequence length
    
    Returns:
    x -- array of shape (m, Tx, len(chars))
    y -- array of shape (m, Tx, len(chars))
    """
    
    m = int((len(text)-Tx-1)/stride)
    x = np.zeros((m, Tx, n_x), dtype=np.bool)
    y = np.zeros((m, Tx, n_x), dtype=np.bool)
    
    for t, i in enumerate(tqdm(range(0,m*stride,stride))):
        fragment=text[i:i+Tx]
        pred=text[i+1:i+Tx+1]
        x[t, :, :] = str_to_vec(fragment)
        
        y[t,:, :] = str_to_vec(pred)
    return x, y 

X,Y=vectorization(names_text, 10, n_x=len(characters), Tx=Tx)
Xf,Yf=vectorization(finnish_names_text, 10, n_x=len(characters), Tx=Tx)
X.shape, Y.shape, Xf.shape, Yf.shape, 


100%|██████████| 15348445/15348445 [04:41<00:00, 54499.81it/s]

100%|██████████| 32156/32156 [00:00<00:00, 52676.88it/s]


((15348445, 25, 41), (15348445, 25, 41), (32156, 25, 41), (32156, 25, 41))

In [9]:
id_=3430
vec_to_str(Xf[id_]),vec_to_str(np.array(Yf[id_]))

('a corp\nrokkem oy\nlabquali', ' corp\nrokkem oy\nlabqualit')

# Model

In [84]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    out = np.random.choice(range(len(characters)), p = probas.ravel())
    out=characters[out]
    return out

def generate_output(model, text_start, length=5):
    
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print(f"Diversity: {diversity}")
        for l in range(length):
            generated = '\n'
            sentence = ('{0:\n>' + str(Tx) + '}').format(text_start).lower()
            generated += text_start 
            gens=0

            generated = ""
            inp=text_start
            sentence = inp

            x_pred = np.zeros((1, Tx, len(characters)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            for pred in preds:
                next_index = sample(pred)
                next_char = next_index
                sentence = sentence + next_char
                generated += next_char
            print(f"{inp}{generated}")
        
generate_output(model,"accent")

Diversity: 0.2
accentdoenseo aeptsuieiunsalotwitnouta
accentpkc tslss o ustrtsau itao urt  v
accent3h stela tlyaiaautseepraa midptc
accentblh fipoeea be1niutttsgtalolnsde
accentlsestef sc eipeiysreysuiteo tspt
Diversity: 0.5
accenttlurtioldce tasiat ucoitfayaw ui
accentpcuntrragia i rksctiaa ftaai ena
accentcioätes g lysift toiiopiuiethtii
accentiteltec eet  n  smmrtoonyugudtoa
accenttaamtol  iaolel toayt esu vtitua
Diversity: 1.0
accentmgelliouya esyilannf oyos iyyrte
accentq obtelsl scsyai  uwabyua oitgit
accentroordhoviisaaptlawvusyn ctoelaao
accentw oxteiyreioaasatpauut ecxaecf s
accentkoorte   m sytsa soe caisay  eso
Diversity: 1.2
accentdrorcuy aire islcfistattouaulatt
accentwpemteea bi iayisiesg ogttti csm
accent1yyrt r l s tcpiifttsionaepotivt
accentmotttea latyesossrusryacoi tirtt
accentyooxtea sumumroilecat eetfaaumpa


# Training

In [25]:
from keras import layers, models
from keras.layers import Dense, Input, LSTM,GRU

drp=0
model=models.Sequential()
model.add(LSTM(128, input_shape=(Tx, len(characters)),return_sequences=True))
model.add(Dense(len(characters),activation="softmax"))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 25, 128)           87040     
_________________________________________________________________
dense_6 (Dense)              (None, 25, 41)            5289      
Total params: 92,329
Trainable params: 92,329
Non-trainable params: 0
_________________________________________________________________


In [26]:
from keras.callbacks import LambdaCallback

def on_epoch_end(epoch, logs):
    generate_output(model,"acc")

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer="adam")

history=model.fit(X, Y, batch_size=4096, validation_split=0.1,  epochs=100, shuffle=True, callbacks=[print_callback])

Train on 13813600 samples, validate on 1534845 samples
Epoch 1/100
13813600/13813600 [==============================] - 659s 48us/step - loss: 2.0369 - val_loss: 1.8503
===Diversity: 0.2 ===
accthtklheti taihcgrit peelc
===Diversity: 0.5 ===
accncye tbirialeaierhs  utlb
===Diversity: 1.0 ===
acclkeutae8lvk e rigiiolvoas
===Diversity: 1.2 ===
accneodoiamildaa ifiuorhtic 
Epoch 2/100
13813600/13813600 [==============================] - 655s 47us/step - loss: 1.8073 - val_loss: 1.7804
===Diversity: 0.2 ===
accnteumteih piho rciirisdet
===Diversity: 0.5 ===
acc tedacezailtotiannaryoe
i
===Diversity: 1.0 ===
acc veunsiirlltm
 udctp isgv
===Diversity: 1.2 ===
accsoautrgbfr aroieaiegeiirj
Epoch 3/100
13813600/13813600 [==============================] - 657s 48us/step - loss: 1.7636 - val_loss: 1.7532
===Diversity: 0.2 ===
acc tluua azasiimsiaryaet
im
===Diversity: 0.5 ===
accckoddo eaminilrihssteairi
===Diversity: 1.0 ===
acc t ssiui iazvtveamestwazs
===Diversity: 1.2 ===
accliosrjestdat jisl

KeyboardInterrupt: 

In [70]:
model.save("alltext.hdf5")

# Per name model

In [107]:
def vectorization(words, n_x, Tx=None):
    """
    Convert X and Y (lists) into arrays to be given to a recurrent neural network.
    
    Arguments:
    X -- 
    Y -- 
    Tx -- integer, sequence length
    
    Returns:
    x -- array of shape (m, Tx, len(chars))
    y -- array of shape (m, Tx, len(chars))
    """
    if Tx is None:
        Tx=len(max(words,key=len))
        print(Tx)
    
    m = len(words)
    x = np.zeros((m, Tx, n_x), dtype=np.bool)
    y = np.zeros((m, Tx, n_x), dtype=np.bool)
    
    for w, word in enumerate(tqdm(words)):
        word=word[:Tx]
        x[w, 0:len(word), :] = str_to_vec(word[:-1],start_with_null=True)
        y[w, 0:len(word),:] = str_to_vec(word)
    return x, y 

Tx=32 # 93 percentile
Xf,Yf=vectorization(finnish_names_arr, n_x=len(characters), Tx=Tx)
Xf.shape, Yf.shape, 


100%|██████████| 17842/17842 [00:00<00:00, 62173.15it/s]


((17842, 32, 41), (17842, 32, 41))

In [108]:
def make_name(model):
    name = []
    x = np.zeros((1, Tx, len(characters)))
    end = False
    i = 0
    
    for i in range(Tx-1):
        probs = list(model.predict(x)[0,i])
        probs = probs / np.sum(probs)
        index = np.random.choice(range(len(characters)), p=probs)
        character = indices_char[index]
        name.append(character)
        x[0, i+1, index] = 1
        i += 1
    
    print(''.join(name))
make_name(model)

tzuconmeg oydreritma oy lagb cl


In [109]:
from keras import layers, models
from keras.layers import Dense, Input, LSTM,GRU

drp=0
model=models.Sequential()
model.add(LSTM(128, input_shape=(Tx, len(characters)),return_sequences=True))
model.add(Dense(len(characters),activation="softmax"))
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 32, 128)           87040     
_________________________________________________________________
dense_10 (Dense)             (None, 32, 41)            5289      
Total params: 92,329
Trainable params: 92,329
Non-trainable params: 0
_________________________________________________________________


In [101]:
from keras.callbacks import LambdaCallback

def on_epoch_end(epoch, logs):
    for i in range(10):
        make_name(model)
        
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)


In [102]:
model.compile(loss='categorical_crossentropy', optimizer="adam")

history=model.fit(Xf, Yf, batch_size=32, validation_split=0.1,  epochs=1000, shuffle=True, callbacks=[print_callback])

Train on 16057 samples, validate on 1785 samples
Epoch 1/1000
16057/16057 [==============================] - 20s 1ms/step - loss: 1.4065 - val_loss: 1.2721
9wipareno pree ravihsa oylus o.
9cenbometitesy oy lanhepden ta.
jwospataigke iingelasespgrepes.
 oyladdiliivlk ituales oyd cy .
häsoluled oymeri foysiolt gou .

tentninxufe gu lnekiintiknnsr.
oevdl fi ylitdh fawunxos cetpe.
xsaltroynlin oyigiind oytpatar.
riregngont oy meiqiateng oy it.
 ivindaj kihdtlo oyokorostiifp.
Epoch 2/1000
16057/16057 [==============================] - 20s 1ms/step - loss: 1.2338 - val_loss: 1.2039
hän funlckapor oystlakkiinet o.
jreaalt oyg maskenh oy thertod.
ärickerstalig oystäk oyshlalic.
nsgedli seskinvarcti ligtiso f.
igrnco solut olistauvshi en oy.
 9prtostveita sutulinc oyitin .
waurasti ark oye oyle yronk oy.
0o8ri estiavoft oysitgijars oy.
kkaren felavostg xed oy ufckn .
9finkoni grb engtöa duchtint o.
Epoch 3/1000
 3552/16057 [=====>........................] - ETA: 14s - loss: 1.2098

KeyboardInterrupt: 